# bw2-snippets
A compilation of code snippets, useful when using brightway2, the python framework for life cycle assessment, especially when starting

The code snippets are compiled in a jupyter notebook, and described with markdown text and/or in-line comments. 

Future to-do:
- split in several notebooks, per theme / user level
- create a jupyter notebook "extension" to access the code snippets directly in jupyter's gui 

## Starter

### import common libaries

In [ ]:
import brightway2 as bw2
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

### select existing bw2 project

In [ ]:
pro = 'my_project_name'
bw2.projects.set_current(pro)

### search for activity in database

In [ ]:
db_name = 'use_1_tree-planting' # name of database to search
# search via list comprehension
act = [a for a in bw2.Database(db_name)
       if "keyword to search" in str(a)
       and not "keyword to exclude" in str(a)
       # add more keywords to search/exclude by adding lines below:  and "..." in str(a)
      ] 
act.sort(reverse=True) # optional sorting of list
act

### select IPCC and ILCD methods, create list of units with corrected syntax

In [ ]:
IPCC = [method for method in bw2.methods if "IPCC 2013" in str(method) 
        and "GWP 100" in str(method) 
        and "LT" not in str(method)
        and "V1" not in str(method)]

ILCDs = [method for method in bw2.methods if "ILCD 2.0" in str(method) 
        and "LT" not in str(method)
        and "V1" not in str(method)
        and "climate change" not in str(method)] # excluding climate change

methods = ILCDs.copy()
methods.append(IPCC[0])
method_units = [bw2.methods[method]['unit'] for method in methods]
method_units[0] = r'mol $H^{+}$-Eq'
method_units[6] = r'kg $U_235$-Eq'
method_units[-5] = r'$m^3$ water'
method_units[-1] = r'kg $CO_2$-Eq'

#methods, method_units

## Contribution analysis